### This notebook contains the code to generate the submission for the "DengAI: Predicting Disease Spread" competition.

The preprocessed data is saved in my prep_data folder.

This notebook trains the model and predicts the submission values. We want to predict the number of cases, rounded to an integer.

The score is evaluated using the mean absolute error metric.

Here we train the XGBoost model. We also train two separate models, one for each city. Additionally we add lag features, and change the model so that it is compatible with those, by using predicted values to predict ones further in the future. Note that the test data start date is just the next date after the training data ends.

In [26]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

In [27]:
X_train_sj = pd.read_csv('prep_data/X_train_prep_sj.csv')
y_train_sj = pd.read_csv('prep_data/y_train_prep_sj.csv')
X_valid_sj = pd.read_csv('prep_data/X_valid_prep_sj.csv')
y_valid_sj = pd.read_csv('prep_data/y_valid_prep_sj.csv')
X_train_iq = pd.read_csv('prep_data/X_train_prep_iq.csv')
y_train_iq = pd.read_csv('prep_data/y_train_prep_iq.csv')
X_valid_iq = pd.read_csv('prep_data/X_valid_prep_iq.csv')
y_valid_iq = pd.read_csv('prep_data/y_valid_prep_iq.csv')

In [28]:
# We start by using just the splitted training data to predict the validation data
# For each day in the validation data, we then need the lag features; since only for the first day those are actually available
# in the training data, we need to predict the first day, then use that predicted value to predict the second day, and so on

# San Juan
lags_list_sj = [1, 2, 3, 5, 6, 9] # from partial autocorrelation function
for lag in lags_list_sj:
    X_train_sj['lag_{}'.format(lag)] = y_train_sj.shift(lag)

# Drop the nan values, those are very early anyway
X_train_sj.dropna(inplace=True)
# Also for y
y_train_sj = y_train_sj.iloc[X_train_sj.index]

X_train_sj.head()

,year,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,...,station_min_temp_c,station_precip_mm,weekofyear_sin,weekofyear_cos,lag_1,lag_2,lag_3,lag_5,lag_6,lag_9
9,1990,0.077686,0.12155,0.160683,0.202567,14.41,300.154286,300.278571,296.651429,302.3,...,24.4,1.1,-0.120537,-0.992709,10.0,5.0,4.0,6.0,3.0,4.0
10,1990,0.192875,0.08235,0.191943,0.152929,22.27,299.512857,299.592857,296.041429,301.8,...,21.7,63.7,-0.239316,-0.970942,6.0,10.0,5.0,2.0,6.0,5.0
11,1990,0.291600,0.21180,0.301200,0.280667,59.17,299.667143,299.750000,296.334286,302.0,...,23.9,12.2,-0.354605,-0.935016,8.0,6.0,10.0,4.0,2.0,4.0
12,1990,0.150567,0.17170,0.226900,0.214557,16.48,299.558571,299.635714,295.960000,301.8,...,22.8,32.6,-0.464723,-0.885456,2.0,8.0,6.0,5.0,4.0,3.0
13,1990,0.048585,0.24715,0.379700,0.381357,32.66,299.862857,299.950000,296.172857,303.0,...,22.8,37.6,-0.568065,-0.822984,6.0,2.0,8.0,10.0,5.0,6.0


In [29]:
# For early stopping to work, make a copy of the validation data with the lag features
# The easiest way to also include the days that are at the end of the training data is to concatenate the training and validation data
# and then split again
X_sj = pd.concat([X_train_sj, X_valid_sj])
y_sj = pd.concat([y_train_sj, y_valid_sj])
for lag in lags_list_sj:
    X_sj['lag_{}'.format(lag)] = y_sj.shift(lag)
X_valid_lagged_sj = X_sj.iloc[len(X_train_sj):]
X_valid_lagged_sj.head()

,year,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,...,station_min_temp_c,station_precip_mm,weekofyear_sin,weekofyear_cos,lag_1,lag_2,lag_3,lag_5,lag_6,lag_9
0,2004,0.040150,0.076940,0.094871,0.139443,52.29,301.100000,301.257143,297.304286,303.3,...,24.4,32.8,-0.992709,-1.205367e-01,10.0,14.0,7.0,14.0,20.0,18.0
1,2004,0.074778,-0.067450,0.225257,0.233200,245.73,299.798571,299.957143,297.081429,302.9,...,23.3,158.2,-1.000000,-1.836970e-16,13.0,10.0,14.0,14.0,14.0,8.0
2,2004,0.033475,0.007625,0.243580,0.178633,10.06,301.028571,301.207143,296.478571,303.6,...,24.4,21.8,-0.992709,1.205367e-01,27.0,13.0,10.0,7.0,14.0,7.0
3,2004,-0.016800,0.025267,0.090471,0.058367,34.73,300.680000,300.678571,296.584286,302.5,...,24.4,24.4,-0.970942,2.393157e-01,13.0,27.0,13.0,14.0,7.0,20.0
4,2004,0.071232,0.093750,0.236029,0.213871,83.20,300.508571,300.678571,296.864286,303.4,...,23.9,41.4,-0.935016,3.546049e-01,18.0,13.0,27.0,10.0,14.0,14.0


In [30]:
# Train the model on the training data
model_sj = XGBRegressor(n_estimators=100, learning_rate=0.1, n_jobs=-1, random_state=42)
model_sj.fit(X_train_sj, y_train_sj)

# Predict the first day; get the corresponding row from the validation data
X_valid_first_sj = X_valid_sj.iloc[[0]]
# Then get the lag features (from y_train)
for lag in lags_list_sj:
    X_valid_first_sj['lag_{}'.format(lag)] = y_train_sj.iloc[-lag].values
    #print(y_train_sj.iloc[-lag].values)

# Predict the first day
y_pred_sj = model_sj.predict(X_valid_first_sj)
print('predicted', y_pred_sj)
print('actual', y_valid_sj.iloc[0].values)
X_valid_first_sj.head()

predicted [14.6932955]
actual [13]


,year,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,...,station_min_temp_c,station_precip_mm,weekofyear_sin,weekofyear_cos,lag_1,lag_2,lag_3,lag_5,lag_6,lag_9
0,2004,0.04015,0.07694,0.094871,0.139443,52.29,301.1,301.257143,297.304286,303.3,...,24.4,32.8,-0.992709,-0.120537,10,14,7,14,20,18


In [31]:
# Now add the predicted value to the training data, and repeat the process
X_train_sj_appended = pd.concat([X_train_sj, X_valid_first_sj], axis=0)
y_train_sj_appended = pd.concat([y_train_sj, pd.DataFrame(y_pred_sj, columns=y_train_sj.columns)], axis=0)
y_train_sj_appended.tail()

,total_cases
744,14.000000
745,7.000000
746,14.000000
747,10.000000
0,14.693295


In [32]:
# So now put the whole thing in a loop
# NOTE that here we train the model ONCE, with just the training data. We could also try adding the predicted values to the training data
# and retrain the model each time
def train_predict_recursively(X_train, y_train, X_valid, X_valid_lagged, y_valid, lags_list, n_est, lr):
    model = XGBRegressor(n_estimators=n_est, learning_rate=lr, n_jobs=-1, early_stopping_rounds=5, random_state=42)
    model.fit(X_train, y_train,
              eval_set=[(X_valid_lagged, y_valid)], # early stopping with lagged validation data!
              verbose=False)
    best_n_est = model.best_iteration
    
    y_preds = []
    for i in range(len(X_valid)):
        X_valid_i = X_valid.iloc[[i]]
        for lag in lags_list:
            X_valid_i['lag_{}'.format(lag)] = y_train.iloc[-lag].values
        y_pred = model.predict(X_valid_i)
        y_preds.append(y_pred)
        
        #X_train = pd.concat([X_train, X_valid_i], axis=0)
        y_train = pd.concat([y_train, pd.DataFrame(y_pred, columns=y_train.columns)], axis=0)
    
    return y_preds, best_n_est

In [33]:
# San Juan test
y_preds_sj, best_n_est = train_predict_recursively(X_train_sj, y_train_sj, X_valid_sj, X_valid_lagged_sj, y_valid_sj, lags_list_sj, 100, 0.1)
print('MAE San Juan:', mean_absolute_error(y_valid_sj, y_preds_sj))

MAE San Juan: 16.540217049578402


In [34]:
# Optimise the hyperparameters
param_grid = {
    'n_estimators': [100, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2]
}

scores = []
for n_est in param_grid['n_estimators']:
    for lr in param_grid['learning_rate']:
        y_preds_sj, best_n_est = train_predict_recursively(X_train_sj, y_train_sj, X_valid_sj, X_valid_lagged_sj, y_valid_sj, lags_list_sj, n_est, lr)
        score = mean_absolute_error(y_valid_sj, y_preds_sj)
        scores.append(score)
        print('n_est:', n_est, 'best n_est:', best_n_est, 'lr:', lr, 'MAE:', score)
print(min(scores))

n_est: 100 best n_est: 99 lr: 0.01 MAE: 25.948349526587954
n_est: 100 best n_est: 70 lr: 0.05 MAE: 16.755308546918503
n_est: 100 best n_est: 33 lr: 0.1 MAE: 16.540217049578402
n_est: 100 best n_est: 18 lr: 0.2 MAE: 16.659987381164065
n_est: 500 best n_est: 395 lr: 0.01 MAE: 16.754468486664145
n_est: 500 best n_est: 70 lr: 0.05 MAE: 16.755308546918503
n_est: 500 best n_est: 33 lr: 0.1 MAE: 16.540217049578402
n_est: 500 best n_est: 18 lr: 0.2 MAE: 16.659987381164065
16.540217049578402


In [35]:
# So 500 est is plenty, check lr around 0.1
param_grid = {
    'n_estimators': [500],
    'learning_rate': [0.08, 0.09, 0.11, 0.12]
}

scores = []
for n_est in param_grid['n_estimators']:
    for lr in param_grid['learning_rate']:
        y_preds_sj, best_n_est = train_predict_recursively(X_train_sj, y_train_sj, X_valid_sj, X_valid_lagged_sj, y_valid_sj, lags_list_sj, n_est, lr)
        score = mean_absolute_error(y_valid_sj, y_preds_sj)
        scores.append(score)
        print('n_est:', n_est, 'best n_est:', best_n_est, 'lr:', lr, 'MAE:', score)
print(min(scores))

n_est: 500 best n_est: 52 lr: 0.08 MAE: 17.141878947298576
n_est: 500 best n_est: 38 lr: 0.09 MAE: 17.056879890725966
n_est: 500 best n_est: 38 lr: 0.11 MAE: 17.161736397033042
n_est: 500 best n_est: 38 lr: 0.12 MAE: 17.053482111464156
17.053482111464156


In [36]:
# So go with 33, 0.1 to train on full data
model_fin_sj = XGBRegressor(n_estimators=33, learning_rate=0.1, n_jobs=-1, random_state=42)
# Get full data
X_full_sj = pd.concat([X_train_sj, X_valid_lagged_sj])
y_full_sj = pd.concat([y_train_sj, y_valid_sj])

model_fin_sj.fit(X_full_sj, y_full_sj)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=33, n_jobs=-1,
             num_parallel_tree=None, random_state=42, ...)

In [37]:
# Next get the same for Iquitos
lags_list_iq = [1, 3, 5] # from partial autocorrelation function

for lag in lags_list_iq:
    X_train_iq['lag_{}'.format(lag)] = y_train_iq.shift(lag)
X_train_iq.dropna(inplace=True)
y_train_iq = y_train_iq.iloc[X_train_iq.index]

# For early stopping
X_iq = pd.concat([X_train_iq, X_valid_iq])
y_iq = pd.concat([y_train_iq, y_valid_iq])
for lag in lags_list_iq:
    X_iq['lag_{}'.format(lag)] = y_iq.shift(lag)
X_valid_lagged_iq = X_iq.iloc[len(X_train_iq):]

# Test model
y_preds_iq, best_n_est = train_predict_recursively(X_train_iq, y_train_iq, X_valid_iq, X_valid_lagged_iq, y_valid_iq, lags_list_iq, 100, 0.1)
print('MAE Iquitos:', mean_absolute_error(y_valid_iq, y_preds_iq))

MAE Iquitos: 6.803161460046585


In [38]:
# Optimise
param_grid = {
    'n_estimators': [100],
    'learning_rate': [0.05, 0.09, 0.1, 0.11, 0.2]
}

scores = []
for n_est in param_grid['n_estimators']:
    for lr in param_grid['learning_rate']:
        y_preds_iq, best_n_est = train_predict_recursively(X_train_iq, y_train_iq, X_valid_iq, X_valid_lagged_iq, y_valid_iq, lags_list_iq, n_est, lr)
        score = mean_absolute_error(y_valid_iq, y_preds_iq)
        scores.append(score)
        print('n_est:', n_est, 'best n_est:', best_n_est, 'lr:', lr, 'MAE:', score)
print(min(scores))

n_est: 100 best n_est: 51 lr: 0.05 MAE: 6.945795705685248
n_est: 100 best n_est: 32 lr: 0.09 MAE: 6.856953372175877
n_est: 100 best n_est: 30 lr: 0.1 MAE: 6.803161460046585
n_est: 100 best n_est: 25 lr: 0.11 MAE: 6.916942698451189
n_est: 100 best n_est: 11 lr: 0.2 MAE: 7.436771156696173
6.803161460046585


In [39]:
# So just 30, 0.1
model_fin_iq = XGBRegressor(n_estimators=30, learning_rate=0.1, n_jobs=-1, random_state=42)
X_full_iq = pd.concat([X_train_iq, X_valid_lagged_iq])
y_full_iq = pd.concat([y_train_iq, y_valid_iq])
model_fin_iq.fit(X_full_iq, y_full_iq)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=30, n_jobs=-1,
             num_parallel_tree=None, random_state=42, ...)

In [40]:
# Read the test data
X_test_sj = pd.read_csv('prep_data/X_test_prep_sj.csv')
X_test_iq = pd.read_csv('prep_data/X_test_prep_iq.csv')
X_test = pd.concat([X_test_sj, X_test_iq], axis=0)  # easier to make output with this

output = pd.DataFrame()
output['city'] = X_test['city']
output['year'] = X_test['year']
output['weekofyear'] = X_test['weekofyear']

X_test_sj.drop(['weekofyear', 'city'], axis=1, inplace=True)
X_test_iq.drop(['weekofyear', 'city'], axis=1, inplace=True)

In [57]:
# Get predictions, adding the lagged values one by one again
y_preds_sj = []
for i in range(len(X_test_sj)):
    X_test_i = X_test_sj.iloc[[i]]
    for lag in lags_list_sj:
        X_test_i['lag_{}'.format(lag)] = y_full_sj.iloc[-lag].values
    y_pred = model_fin_sj.predict(X_test_i)
    y_preds_sj.append(y_pred)
    y_full_sj = pd.concat([y_full_sj, pd.DataFrame(y_pred, columns=y_full_sj.columns)], axis=0)

y_preds_iq = []
for i in range(len(X_test_iq)):
    X_test_i = X_test_iq.iloc[[i]]
    for lag in lags_list_iq:
        X_test_i['lag_{}'.format(lag)] = y_full_iq.iloc[-lag].values
    y_pred = model_fin_iq.predict(X_test_i)
    y_preds_iq.append(y_pred)
    y_full_iq = pd.concat([y_full_iq, pd.DataFrame(y_pred, columns=y_full_iq.columns)], axis=0)

In [58]:
# Round
y_preds_sj = [int(value.round()) for value in y_preds_sj]
y_preds_iq = [int(value.round()) for value in y_preds_iq]

# First make an empty column
output['total_cases'] = pd.Series()
# Assign for each city
output.loc[output['city'] == 'sj', 'total_cases'] = y_preds_sj
output.loc[output['city'] == 'iq', 'total_cases'] = y_preds_iq

output.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,12
1,sj,2008,19,11
2,sj,2008,20,13
3,sj,2008,21,14
4,sj,2008,22,14


In [59]:
# Save to csv
output.to_csv('submissions/submission_XGB_time.csv', index=False)

### Final note:

This model gets a submission score of 28.4639. That's worse than without lag features.